In [1]:
import skimage
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
import numpy as np
import os
import scipy.ndimage.morphology
from skimage.measure import label, regionprops
from skimage import feature
from skimage import color
from skimage import filters
import cv2
from sklearn.cluster import KMeans, MiniBatchKMeans
import sklearn
import csv

# Load images

In [2]:
PATH_TO_RESOURCES='../data/images/'
file_paths = !ls $PATH_TO_RESOURCES*.jpg | sort
imgs = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY) for path in file_paths]

# Descriptors

In [3]:
desc = []
sift = cv2.xfeatures2d.SIFT_create()

for im in imgs:
    kp, d = sift.detectAndCompute(im, None)
    if d is not None:
        desc.append(d.astype(np.float32))
desc = np.asarray(desc)


In [4]:
train_desc = np.vstack(desc)
print(train_desc.shape)

(48656, 128)


In [5]:
train_mean = np.mean(train_desc, axis=1, keepdims=True)
train_desc = train_desc - train_mean

In [6]:
train_cov = np.dot(train_desc.T, train_desc)
eigvals, eigvecs = np.linalg.eig(train_cov)
perm = eigvals.argsort()                   # sort by increasing eigenvalue
pca_transform = eigvecs[:, perm[64:128]]   # eigenvectors for the 64 last eigenvalues
pca_transform.shape, pca_transform.dtype

((128, 64), dtype('float32'))

In [7]:
train_desc = np.dot(train_desc, pca_transform)

In [8]:
train_desc.shape

(48656, 64)

# K-means

In [9]:
kmeans = MiniBatchKMeans(n_clusters=512, random_state=0).fit(train_desc)

/usr/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1558: RuntimeWarning: init_size=300 should be larger than k=512. Setting it to 3*k
  init_size=init_size)
/usr/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1558: RuntimeWarning: init_size=300 should be larger than k=512. Setting it to 3*k
  init_size=init_size)
/usr/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1558: RuntimeWarning: init_size=300 should be larger than k=512. Setting it to 3*k
  init_size=init_size)


In [10]:
kmeans.cluster_centers_.shape

(512, 64)

# Split training set

In [11]:
all_images = []
labels = []
with open('../data/gt_img.csv', newline='') as csvfile:
    for line in csvfile:
        line = line.strip()
        im_name, y = line.split(',')
        all_images.append(im_name)
        labels.append(int(y))
len(all_images), len(labels)

(224, 224)

In [12]:
x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(all_images,
                                                                          labels,
                                                                         test_size=0.2,
                                                                         random_state=42)
len(x_train), len(x_val), len(y_train), len(y_val)

(179, 45, 179, 45)

# Training

In [13]:
l2_normalizer = sklearn.preprocessing.Normalizer(norm='l2', copy=True)
l1_normalizer = sklearn.preprocessing.Normalizer(norm='l1', copy=True)

In [14]:
def compute_descriptors(filelist, train_mean, pca_transform, kmeans):
    image_descriptors = np.zeros((len(filelist), kmeans.n_clusters), dtype=np.float32)
    sift = cv2.xfeatures2d.SIFT_create()
    for ii, file in enumerate(filelist):
        print(file)
        print("Indexing %s" % (file,))
        im = cv2.cvtColor(cv2.imread("../data/images/" + file + ".jpg"), cv2.COLOR_BGR2GRAY)
        # read the descriptors
        kp, desc = sift.detectAndCompute(im, None)
        if desc is None:
            # let the descriptor be 0 for all values
            # note that this is bad and the element should be dropped from the index
            print("WARNING: zero descriptor for %s" % (file,))
            continue
        
        # convert to float
        desc = desc.astype(np.float32)
        
        # center and apply PCA transform
        desc = desc - np.mean(desc, axis=1, keepdims=True)
        desc = np.dot(desc, pca_transform)
        
        # get cluster ids
        clabels = kmeans.predict(desc)
        # compute histogram
        descr_hist = np.histogram(clabels, bins=kmeans.n_clusters)[0]
        
        descr_hist = np.reshape(descr_hist, (1, -1))
        # l1 norm
        descr_hist = l1_normalizer.transform(descr_hist)
        
        # take the sqrt (Hellinger kernel)
        descr_hist = np.sqrt(descr_hist)
        
        # l2 norm
        descr_hist = l2_normalizer.transform(descr_hist)
        
        # update the index
        image_descriptors[ii] = descr_hist
    print("Indexing complete.")
    return image_descriptors

In [ ]:
x_train_desc, x_val_desc = [compute_descriptors(filelist, train_mean, pca_transform, kmeans) 
    for filelist in (x_train, x_val)]

img_0073149
Indexing img_0073149
img_0000068
Indexing img_0000068
img_0000077
Indexing img_0000077
img_0073104
Indexing img_0073104
img_0073140
Indexing img_0073140
img_0000093
Indexing img_0000093
img_0073209
Indexing img_0073209
img_0073102
Indexing img_0073102
img_0073128
Indexing img_0073128


In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10000,
                              random_state=0)
#clf = svm.SVC()

In [ ]:
clf.fit(x_train_desc, y_train)

In [ ]:
clf.score(x_val_desc, y_val)

In [ ]:
y_pred = clf.predict(x_val_desc)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_val, y_pred))

# Submission

In [ ]:
PATH_TO_TEST='../data/imgs_test/'
test_files = !ls $PATH_TO_TEST*.jpg | sort

In [ ]:
x_test_desc = compute_descriptors(test_files, train_mean, pca_transform, kmeans)